In [22]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import os
from datetime import datetime
import uuid

# Load the YOLOv8 model
model = YOLO('best.pt')

# Open the video file
#video_path = "C:\\Users\\soumy\\Downloads\\videoplayback.mp4"
video_path = "pexels-anastasia-shuraeva-7671385 (2160p).mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Create a folder for saving tracked information with current date and time
output_folder_root = "tracked_info"
output_folder_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs(output_folder_root, exist_ok=True)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, conf=0.3, iou=0.5, tracker="bytetrack.yaml")
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            confidences = results[0].boxes.conf.cpu().numpy().astype(int)
            class_names = results[0].names
        # Get the boxes and track IDs
        # boxes = results[0].boxes.xywh.cpu()
        # track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks and save cropped images with random filenames
        for box, track_id, class_name in zip(boxes, track_ids, class_names):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

            # Crop the region of interest
            roi = frame[y:h, x:w]

            img_count = len(track)
            if img_count < 60:
                # Save the cropped image with a random unique filename
                unique_filename = str(uuid.uuid4())[:8]  # Using a portion of a UUID as a unique identifier
                tracker_folder = os.path.join(output_folder_root, output_folder_datetime, f"tracker_{track_id}")
                os.makedirs(tracker_folder, exist_ok=True)
                filename = f"{unique_filename}.jpg"
                output_path = os.path.join(tracker_folder, filename)
                cv2.imwrite(output_path, roi)

        # Display the annotated frame
        cv2.imshow("Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



0: 384x640 3 ears, 3 eyes, 3 faces, 2 mouths, 177.1ms
Speed: 2.0ms preprocess, 177.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 ears, 3 eyes, 3 faces, 2 mouths, 126.9ms
Speed: 2.0ms preprocess, 126.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 ears, 3 eyes, 3 faces, 2 mouths, 109.3ms
Speed: 1.7ms preprocess, 109.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ear, 3 eyes, 4 faces, 3 mouths, 110.0ms
Speed: 1.0ms preprocess, 110.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ear, 3 eyes, 4 faces, 4 mouths, 111.6ms
Speed: 2.9ms preprocess, 111.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ear, 4 eyes, 4 faces, 4 mouths, 105.9ms
Speed: 2.9ms preprocess, 105.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ear, 4 eyes, 4 faces, 4 mouths, 118.8ms
Speed: 2.0ms preprocess, 118.8m

In [23]:
import os

folder_path = 'tracked_info'

# Get a list of all subfolders in the folder
all_folders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

# Find the most recently modified folder
most_recent_folder = max(all_folders, key=os.path.getmtime)

# Extract the name of the most recently modified folder
most_recent_folder_name = os.path.basename(most_recent_folder)

# Count the number of subfolders in the most recently modified folder
number_of_classes = len([f for f in os.scandir(most_recent_folder) if f.is_dir()])

# Now you can use the variable `most_recent_folder` for the path,
# `most_recent_folder_name` for the name, and `number_of_folders` for the count of subfolders.
print("Most recently modified folder path:", most_recent_folder)
print("Most recently modified folder name:", most_recent_folder_name)
print("Number of folders inside:", number_of_classes)


Most recently modified folder path: C:\Users\soumy\tracked_info\20231230_125852
Most recently modified folder name: 20231230_125852
Number of folders inside: 16


In [24]:
import os
import shutil
import random

folder_path = 'tracked_info'

# Get a list of all subfolders in the folder
all_folders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

# Find the most recently modified folder
most_recent_folder = max(all_folders, key=os.path.getmtime)
most_recent_folder_name = os.path.basename(most_recent_folder)

# Create a new folder in the same directory with the name most_recent_folder_name_test
new_folder_path = os.path.join(os.path.dirname(most_recent_folder), f"{most_recent_folder_name}_test")
os.makedirs(new_folder_path, exist_ok=True)

# Count the number of subfolders in the most recently modified folder
number_of_folders = len([f for f in os.scandir(most_recent_folder) if f.is_dir()])

# Iterate through each subfolder and copy 1/4 of its content to the new folder
for subfolder in os.scandir(most_recent_folder):
    if subfolder.is_dir():
        subfolder_name = os.path.basename(subfolder.path)
        new_subfolder_path = os.path.join(new_folder_path, subfolder_name)
        os.makedirs(new_subfolder_path, exist_ok=True)

        # List all files in the subfolder
        files_in_subfolder = [f.path for f in os.scandir(subfolder.path) if f.is_file()]

        # Randomly select 1/4 of the files
        selected_files = random.sample(files_in_subfolder, k=len(files_in_subfolder) // 4)

        # Copy selected files to the new subfolder
        for file_path in selected_files:
            shutil.copy(file_path, new_subfolder_path)

# Now you have a new folder with the name most_recent_folder_name_test
# containing a subset of files from each subfolder in the most recently modified folder
print(f"Subset of files from each subfolder in {most_recent_folder_name} copied to {new_folder_path}")


Subset of files from each subfolder in 20231230_125852 copied to C:\Users\soumy\tracked_info\20231230_125852_test


In [25]:
!nvidia-smi

Sat Dec 30 13:02:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8               5W / 130W |    425MiB /  8188MiB |     47%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
#from tensorflow.keras.backend import set_session
tf.__version__
input_shape = (256, 256, 3)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
training_set = train_datagen.flow_from_directory(
    most_recent_folder,
    target_size=(256, 256),  # Larger target size
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(new_folder_path,
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical')
class_names = list(training_set.class_indices.keys())
for i in range(len(class_names)):
    print(f"Class {i}: {class_names[i]}")
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
#set_session(tf.compat.v1.Session(config=config))
# Create a more complex CNN model
cnn = tf.keras.models.Sequential()

# Convolutional layer 1
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Convolutional layer 2
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Convolutional layer 3
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Convolutional layer 4
cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Flatten layer
cnn.add(tf.keras.layers.Flatten())

# Fully connected layer 1
cnn.add(tf.keras.layers.Dense(units=1024, activation='relu'))

# Fully connected layer 2
cnn.add(tf.keras.layers.Dense(units=512, activation='relu'))

# Output layer
cnn.add(tf.keras.layers.Dense(units=number_of_classes, activation='softmax'))

# Compile the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
cnn.summary()
#cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Found 549 images belonging to 16 classes.
Found 129 images belonging to 16 classes.
Class 0: tracker_1
Class 1: tracker_10
Class 2: tracker_2
Class 3: tracker_26
Class 4: tracker_27
Class 5: tracker_28
Class 6: tracker_3
Class 7: tracker_33
Class 8: tracker_4
Class 9: tracker_42
Class 10: tracker_43
Class 11: tracker_5
Class 12: tracker_56
Class 13: tracker_61
Class 14: tracker_62
Class 15: tracker_63
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 127, 127, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 125, 125, 128)     73856     
                                             

In [27]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
18/18 [==============================] - 29s 2s/step - loss: 2.8645 - accuracy: 0.1002 - val_loss: 2.4679 - val_accuracy: 0.2093
Epoch 2/25
18/18 [==============================] - 27s 1s/step - loss: 2.1896 - accuracy: 0.3042 - val_loss: 1.2883 - val_accuracy: 0.7054
Epoch 3/25
18/18 [==============================] - 27s 2s/step - loss: 1.1442 - accuracy: 0.6576 - val_loss: 0.2781 - val_accuracy: 0.9457
Epoch 4/25
18/18 [==============================] - 28s 2s/step - loss: 0.4245 - accuracy: 0.8525 - val_loss: 0.1103 - val_accuracy: 0.9690
Epoch 5/25
18/18 [==============================] - 28s 2s/step - loss: 0.2082 - accuracy: 0.9308 - val_loss: 0.0212 - val_accuracy: 0.9922
Epoch 6/25
18/18 [==============================] - 28s 2s/step - loss: 0.1932 - accuracy: 0.9435 - val_loss: 0.0364 - val_accuracy: 1.0000
Epoch 7/25
18/18 [==============================] - 28s 2s/step - loss: 0.1152 - accuracy: 0.9636 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 8/25
18/18 [==

In [28]:
cnn.save('simple_model.h5')

c:\Users\soumy\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [36]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Check if GPU is available for TensorFlow operations
print("GPU Available: ", tf.test.is_gpu_available())


Num GPUs Available:  0
GPU Available:  False


In [35]:
import tensorflow as tf

# Explicitly set GPU memory growth to True
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


In [30]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import os
from datetime import datetime
import uuid

# Load the YOLOv8 model
model = YOLO('best.pt')

# Open the video file
#video_path = "C:\\Users\\soumy\\Downloads\\videoplayback.mp4"
video_path = "pexels-drvivasayam-youtube-channel-6733515 (1080p).mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history and image count for each track ID
track_history = defaultdict(lambda: {'history': [], 'img_count': 0})

# Create a folder for saving tracked information with current date and time
output_folder_root = "valid_info"
output_folder_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs(output_folder_root, exist_ok=True)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, conf=0.3, iou=0.5, tracker="bytetrack.yaml")
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            confidences = results[0].boxes.conf.cpu().numpy().astype(int)
            class_names = results[0].names
        # Get the boxes and track IDs
        # boxes = results[0].boxes.xywh.cpu()
        # track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks and save cropped images with random filenames
        for box, track_id, class_name in zip(boxes, track_ids, class_names):
            x, y, w, h = box
            track_info = track_history[track_id]
            track = track_info['history']
            img_count = track_info['img_count']

            track.append((float(x), float(y)))  # x, y center point

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

            # Crop the region of interest
            roi = frame[y:h, x:w]

            if img_count < 20:
                # Save the cropped image with a random unique filename
                unique_filename = str(uuid.uuid4())[:8]  # Using a portion of a UUID as a unique identifier
                tracker_folder = os.path.join(output_folder_root, output_folder_datetime, f"tracker_{track_id}")
                os.makedirs(tracker_folder, exist_ok=True)
                filename = f"{unique_filename}.jpg"
                output_path = os.path.join(tracker_folder, filename)
                cv2.imwrite(output_path, roi)

                # Update the image count for the current track ID
                track_info['img_count'] += 1

        # Display the annotated frame
        cv2.imshow("Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



0: 384x640 3 ears, 3 eyes, 3 faces, 2 mouths, 161.4ms
Speed: 7.5ms preprocess, 161.4ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 ears, 3 eyes, 3 faces, 2 mouths, 148.9ms
Speed: 2.0ms preprocess, 148.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 ears, 3 eyes, 3 faces, 2 mouths, 125.0ms
Speed: 2.3ms preprocess, 125.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ear, 3 eyes, 4 faces, 3 mouths, 124.2ms
Speed: 1.1ms preprocess, 124.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ear, 3 eyes, 4 faces, 4 mouths, 130.9ms
Speed: 1.0ms preprocess, 130.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ear, 4 eyes, 4 faces, 4 mouths, 136.2ms
Speed: 2.0ms preprocess, 136.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ear, 4 eyes, 4 faces, 4 mouths, 173.2ms
Speed: 2.0ms preprocess, 173.2m

In [29]:
loaded_model = load_model('simple_model.h5')

In [34]:
import os
import tensorflow as tf
from tensorflow.compat.v1.losses import sparse_softmax_cross_entropy
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from datetime import datetime
# Path to the "valid_info" directory
valid_info_path = 'valid_info'

input_shape = (256, 256, 3)
confidence_threshold = .9
valid_percentage_threshold = 0.7
# Function to get the most recently modified subdirectory
def get_most_recent_subdirectory(directory):
    subdirectories = [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    return max(subdirectories, key=os.path.getmtime)

# Get the most recently modified subdirectory within "valid_info"
most_recent_folder = get_most_recent_subdirectory(valid_info_path)

class_indices = training_set.class_indices
class_names = list(class_indices.keys())

# Loop through folders in the most recently modified subdirectory
for folder_name in os.listdir(most_recent_folder):
    folder_path = os.path.join(most_recent_folder, folder_name)

    # Check if the path is a directory
    if os.path.isdir(folder_path):
        print(f"\nProcessing images in folder: {folder_name}")

        # Initialize counters for valid and total predictions
        valid_count = 0
        total_count = 0

        # Loop through images in the current folder
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)

            # Load and preprocess the image for prediction
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0  # Normalize the pixel values to be between 0 and 1

            # Make a prediction using the loaded model
            predictions = loaded_model.predict(img_array)

            # Get the predicted class index and confidence
            predicted_class_index = np.argmax(predictions)
            confidence = predictions[0][predicted_class_index]

            # Print "valid" or "invalid" based on the confidence threshold
            result = "valid" if confidence > confidence_threshold else "invalid"

            # Display class name instead of class index
            predicted_class_name = class_names[predicted_class_index]

            print(f"Image: {img_name}, Predicted Class: {predicted_class_name}, Confidence: {confidence:.4f}, Result: {result}")

            # Update counters
            if result == "valid":
                valid_count += 1
            total_count += 1

        # Check if more than 80% of the images are valid in the folder
        valid_percentage = valid_count / total_count
        folder_result = "valid" if valid_percentage > valid_percentage_threshold else "invalid"

        print(f"Folder: {folder_name}, Valid Percentage: {valid_percentage:.2%}, Result: {folder_result}")


Processing images in folder: tracker_1
1/1 [==============================] - 0s 39ms/step
Image: 0af984a8.jpg, Predicted Class: tracker_1, Confidence: 0.9983, Result: valid
1/1 [==============================] - 0s 42ms/step
Image: 25dec763.jpg, Predicted Class: tracker_1, Confidence: 0.9923, Result: valid
1/1 [==============================] - 0s 38ms/step
Image: 489487b7.jpg, Predicted Class: tracker_1, Confidence: 0.9995, Result: valid
1/1 [==============================] - 0s 36ms/step
Image: 66708dda.jpg, Predicted Class: tracker_1, Confidence: 0.9974, Result: valid
1/1 [==============================] - 0s 37ms/step
Image: 66f9ba5c.jpg, Predicted Class: tracker_1, Confidence: 0.9974, Result: valid
1/1 [==============================] - 0s 37ms/step
Image: 69d320e2.jpg, Predicted Class: tracker_1, Confidence: 0.9967, Result: valid
1/1 [==============================] - 0s 39ms/step
Image: 77d0b771.jpg, Predicted Class: tracker_1, Confidence: 0.9998, Result: valid
1/1 [==========